# Simulate a user accessing scenes to estimate # of sectors needed to unseal

## Imports

In [1]:
import random

## Landsat scenes (Rows/Paths) of the US

# Figure out the rows and paths that are needed to cover the US
US_cell_dict = {
    cell_1: {row: 1
             path: 1
             sector: 1}
    cell_2: {row: 30
             path: 30
             sector: 2}
}

## US Metropolitan Statistical Areas (MSAs)

In [ ]:
# The number of Landsat scenes intersected by MSA
MSA_dict = {
    'MSA1': ['cell1', 'cell2', 'cell3'],
    'MSA2': ['cell4', 'cell5'],
}

## User Access Settings

In [ ]:
scenes_requested = random.randint(1, 5)

## Sector and Cache Settings

In [ ]:
hot_storage_cache_time = 1000 # Number of requests (loops) before hot storage cache is cleared
